# Priyanshu Maurya Cassandra Assignment

In [2]:
from cassandra.cluster import Cluster
import pandas as pd


In [3]:
url = 'https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv'
df = pd.read_csv(url)
df.head()


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,UnitsSold,UnitPrice,UnitCost,TotalRevenue,TotalCost,TotalProfit
0,Sub-Saharan Africa,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
1,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
2,Australia and Oceania,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.89,364.69,151880.40,131288.40,20592.00
3,Sub-Saharan Africa,Djibouti,Clothes,Offline,H,5/17/2017,880811536,7/2/2017,562,109.28,35.84,61415.36,20142.08,41273.28
4,Europe,Slovakia,Beverages,Offline,L,10/26/2016,174590194,12/4/2016,3973,47.45,31.79,188518.85,126301.67,62217.18


In [4]:

cluster = Cluster(['localhost'])  
session = cluster.connect()

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sales_data 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

session.set_keyspace('sales_data')


Traceback (most recent call last):
  File "C:\Users\princ\anaconda3\lib\site-packages\cassandra\cluster.py", line 3577, in _reconnect_internal
    return self._try_connect(host)
  File "C:\Users\princ\anaconda3\lib\site-packages\cassandra\cluster.py", line 3599, in _try_connect
    connection = self._cluster.connection_factory(host.endpoint, is_control_connection=True)
  File "C:\Users\princ\anaconda3\lib\site-packages\cassandra\cluster.py", line 1670, in connection_factory
    return self.connection_class.factory(endpoint, self.connect_timeout, *args, **kwargs)
  File "C:\Users\princ\anaconda3\lib\site-packages\cassandra\connection.py", line 846, in factory
    conn = cls(endpoint, *args, **kwargs)
  File "C:\Users\princ\anaconda3\lib\site-packages\cassandra\io\asyncorereactor.py", line 353, in __init__
    self._connect_socket()
  File "C:\Users\princ\anaconda3\lib\site-packages\cassandra\connection.py", line 951, in _connect_socket
    raise socket.error(sockerr.errno, "Tried connec

In [10]:

session.execute('DROP TABLE IF EXISTS bronze_sales')

session.execute("""
    CREATE TABLE IF NOT EXISTS bronze_sales (
        region text,
        country text,
        item_type text,
        sales_channel text,
        order_priority text,
        order_date date,
        order_id int PRIMARY KEY,
        ship_date date,
        units_sold int,
        unit_price float,
        unit_cost float,
        total_revenue float,
        total_cost float,
        total_profit float
    )
""")


In [13]:
from cassandra.query import SimpleStatement

insert_bronze = session.prepare("""
    INSERT INTO bronze_sales (region, country, item_type, sales_channel, order_priority,
                              order_date, order_id, ship_date, units_sold, unit_price, unit_cost,
                              total_revenue, total_cost, total_profit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y').dt.date
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y').dt.date

df[['Order Date', 'Ship Date']].head()

for index, row in df.iterrows():
    session.execute(insert_bronze, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], row['Order ID'], row['Ship Date'], row['UnitsSold'], row['UnitPrice'], 
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))


In [14]:
session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_region (
        region text PRIMARY KEY,
        total_sales float
    )
""")


In [15]:
session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_country (
        country text PRIMARY KEY,
        total_sales float
    )
""")


In [16]:
session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_item_type (
        item_type text PRIMARY KEY,
        total_sales float
    )
""")


In [17]:
from cassandra.query import SimpleStatement

insert_bronze = session.prepare("""
    INSERT INTO bronze_sales (region, country, item_type, sales_channel, order_priority,
                              order_date, order_id, ship_date, units_sold, unit_price, unit_cost,
                              total_revenue, total_cost, total_profit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

for index, row in df.iterrows():
    session.execute(insert_bronze, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], row['Order ID'], row['Ship Date'], row['UnitsSold'], row['UnitPrice'], 
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))


In [19]:
session.execute('DROP TABLE IF EXISTS silver_sales')

session.execute("""
    CREATE TABLE IF NOT EXISTS silver_sales (
        region text,
        country text,
        item_type text,
        sales_channel text,
        order_priority text,
        order_date date,
        order_id int PRIMARY KEY,
        ship_date date,
        units_sold int,
        unit_price float,
        unit_cost float,
        total_revenue float,
        total_cost float,
        total_profit float
    )
""")


In [20]:
insert_silver = session.prepare("""
    INSERT INTO silver_sales (region, country, item_type, sales_channel, order_priority,
                              order_date, order_id, ship_date, units_sold, unit_price, unit_cost,
                              total_revenue, total_cost, total_profit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

for index, row in df.iterrows():
    session.execute(insert_silver, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], row['Order ID'], row['Ship Date'], row['UnitsSold'], row['UnitPrice'], 
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))


In [21]:
region_sales = df.groupby('Region')['TotalRevenue'].sum()

for region, total_sales in region_sales.items():
    session.execute("""
        INSERT INTO gold_sales_region (region, total_sales)
        VALUES (%s, %s)
    """, (region, total_sales))

country_sales = df.groupby('Country')['TotalRevenue'].sum()
for country, total_sales in country_sales.items():
    session.execute("""
        INSERT INTO gold_sales_country (country, total_sales)
        VALUES (%s, %s)
    """, (country, total_sales))

item_sales = df.groupby('Item Type')['TotalRevenue'].sum()
for item_type, total_sales in item_sales.items():
    session.execute("""
        INSERT INTO gold_sales_item_type (item_type, total_sales)
        VALUES (%s, %s)
    """, (item_type, total_sales))


In [22]:
rows = session.execute('SELECT * FROM gold_sales_region')

for row in rows:
    print(row)


Row(region='Australia and Oceania', total_sales=10711258.0)
Row(region='Europe', total_sales=34964748.0)
Row(region='Middle East and North Africa', total_sales=24765128.0)
Row(region='Central America and the Caribbean', total_sales=17570836.0)
Row(region='Asia', total_sales=28840812.0)
Row(region='Sub-Saharan Africa', total_sales=24225438.0)
Row(region='North America', total_sales=3611757.5)


In [23]:

rows = session.execute('SELECT * FROM gold_sales_country')

for row in rows:
    print(row)


Row(country='Malaysia', total_sales=434357.3125)
Row(country='Israel', total_sales=223442.046875)
Row(country='Serbia', total_sales=802989.4375)
Row(country='Djibouti', total_sales=61415.359375)
Row(country='Egypt', total_sales=130261.7578125)
Row(country='Nicaragua', total_sales=5944506.0)
Row(country='Romania', total_sales=1726589.375)
Row(country='Sao Tome and Principe', total_sales=301612.8125)
Row(country='Liberia', total_sales=337990.71875)
Row(country='Vanuatu', total_sales=1222089.25)
Row(country='Morocco', total_sales=503890.09375)
Row(country='Lebanon', total_sales=3699975.25)
Row(country='Madagascar', total_sales=339860.8125)
Row(country='Mali', total_sales=2884921.5)
Row(country='Togo', total_sales=2101183.25)
Row(country='Sri Lanka', total_sales=12866.0703125)
Row(country='Canada', total_sales=3263260.75)
Row(country='Vietnam', total_sales=652532.3125)
Row(country='Mauritius ', total_sales=603225.625)
Row(country='Tanzania', total_sales=3555764.5)
Row(country='Tunisia ', t

In [24]:
rows = session.execute('SELECT * FROM gold_sales_item_type')

for row in rows:
    print(row)


Row(item_type='Household', total_sales=38519084.0)
Row(item_type='Office Supplies', total_sales=27880904.0)
Row(item_type='Vegetables', total_sales=1135114.125)
Row(item_type='Snacks', total_sales=2193642.75)
Row(item_type='Personal Care', total_sales=3191147.75)
Row(item_type='Meat', total_sales=21278866.0)
Row(item_type='Fruits', total_sales=615033.625)
Row(item_type='Beverages', total_sales=2145024.75)
Row(item_type='Cereal', total_sales=9416123.0)
Row(item_type='Cosmetics', total_sales=28727100.0)
Row(item_type='Baby Food', total_sales=5200564.0)
Row(item_type='Clothes', total_sales=4387373.5)
